### Testing on the looping procedure

In [15]:
import streamlit as st
import warnings
import locale
locale.setlocale(locale.LC_ALL, 'en_GB.UTF-8')
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import numpy_financial as npf
import requests
import math
from loan_amortization import loan_amortization, loan_amortization_custom_payment
from parameter import *

In [12]:
import streamlit as st
import warnings
import locale
locale.setlocale(locale.LC_ALL, 'en_GB.UTF-8')
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import numpy_financial as npf
import requests
import math
from loan_amortization import loan_amortization, loan_amortization_custom_payment
from parameter import *



class Calculator():
    def __init__(self):
        
        # COMMENT THIS IF FLASK CONNECTION IS WORKING
        self.cpi = 0.12
        self.markup_percentage = 0.001
        self.maintenance_ratio = 0.08
        self.warranty_rate = 0.05
        self.insurance_rate = 0.015
        self.travel_labor_cost = 300
        self.business_con_rate = 0.02
        self.monthly_interest_rate = self.cpi/12
        # self.monthly_interest_rate = ((1+self.cpi)**(1/12))-1
        
        # UNCOMMENT THIS IF FLASK CONNECTION IS WORKING
        response = requests.get("http://127.0.0.1:5000/get_parameters")
        if response.status_code == 200:
            params = response.json()
            self.cpi = params['cpi']
            self.markup_percentage = params['markup_percentage']
            self.maintenance_ratio = params['maintenance_ratio']
            self.warranty_rate = params['warranty_rate']
            self.insurance_rate = params['insurance_rate']
            self.travel_labor_cost = params['travel_labor_cost']
            self.business_con_rate = params['business_con_rate']
        else:
            st.error("Failed to load parameters!")
            
        self.name = None
        self.monthlyPayment = None
        self.totalPayment = None
        self.invoice = None
                 


    def getMonthlyPayment(self, EquipmentPrice, LoanTerm, terminal_rate, warranty_yrs, insurance='Yes', maintenance='Yes', extra_warranty=0, bussiness_con='Yes'): 
        markup_price = EquipmentPrice # as it has been mark upped in the input form
        principal = markup_price
        # maintenance_fee = (markup_price * self.maintenance_ratio if markup_price > 2500 else 0) if maintenance == 'Yes' else 0 
        maintenance_fee = (markup_price * self.maintenance_ratio) if maintenance == 'Yes' else 0 
 
        # warranty_yrs = 1 if markup_price <= 2000 else 2 if markup_price <= 5000 else 3 if markup_price <= 10000 else 5 if markup_price <= 30000 else 10
        additional_warranty = extra_warranty
        warranty_fee = markup_price * self.warranty_rate * (additional_warranty)
        insurance_fee = markup_price * self.insurance_rate * (warranty_yrs + additional_warranty) if insurance == 'Yes' else 0
        travel_labor_cost = self.travel_labor_cost * LoanTerm
        business_con_fee = (markup_price * self.business_con_rate * (warranty_yrs + additional_warranty) if insurance == 'Yes' else 0) if bussiness_con == 'Yes' else 0
        
        
        total_added_value_services = maintenance_fee + warranty_fee + insurance_fee + business_con_fee + travel_labor_cost
        total_payment = total_added_value_services + principal
        
        terminal_value = markup_price * terminal_rate 
        
        
        monthly_interest_rate = self.monthly_interest_rate
        monthly_added_value_payment = total_added_value_services / (LoanTerm*12)
        monthlyPayment = npf.pmt(monthly_interest_rate, LoanTerm*12, -principal) + monthly_added_value_payment

        self.monthlyPayment = monthlyPayment
        
        results = {
            "principal": principal,
            "total_payment": total_payment, # principal + total_added_value_services
            'total_added_value': total_added_value_services,
            "monthlyPayment": monthlyPayment,
            "maintenance_fee": maintenance_fee,
            "warranty_fee": warranty_fee,
            "insurance_fee": insurance_fee,
            "business_con_fee": business_con_fee,
            "terminal_value": terminal_value,
            'travel_labor_cost': travel_labor_cost
        }

        return results
    
    def getLoanTerm(self, EquipmentPrice, monthlyPayment, terminal_rate, warranty_yrs, insurance='Yes', maintenance='Yes', extra_warranty=0, bussiness_con='Yes'): 
        # Calculating the markup price (principal)
        markup_price = EquipmentPrice  # as it has been marked up in the input form
        principal = markup_price
        
        # Calculating the maintenance fee
        maintenance_fee = (markup_price * self.maintenance_ratio) if maintenance == 'Yes' else 0 
        
        # Calculating the warranty fee
        additional_warranty = extra_warranty
        warranty_fee = markup_price * self.warranty_rate * (additional_warranty)
        
        # Calculating the insurance fee
        insurance_fee = markup_price * self.insurance_rate * (warranty_yrs + additional_warranty) if insurance == 'Yes' else 0
        
        # Travel labor cost
        travel_labor_cost = self.travel_labor_cost
        
        # Calculating the terminal value
        terminal_value = markup_price * terminal_rate
        
        # Calculating the business continuity fee
        business_con_fee = markup_price * self.business_con_rate * (warranty_yrs + additional_warranty) if bussiness_con == 'Yes' else 0

        # Total added value services (maintenance, insurance, etc.), NOT affected by interest
        total_added_value_services = maintenance_fee + warranty_fee + insurance_fee + business_con_fee + travel_labor_cost
        
        # Define monthly interest rate
        monthly_interest_rate = self.monthly_interest_rate
        
        # First, calculate the LoanTerm in months based on the principal only
        LoanTerm_months = npf.nper(monthly_interest_rate, -monthlyPayment, principal, 0)
        
        # Round up to the nearest whole number of months
        LoanTerm_months_rounded = math.ceil(LoanTerm_months)
        
        # Now, calculate the portion of the monthly payment that covers the added value services
        monthly_added_value_payment = total_added_value_services / LoanTerm_months_rounded
        
        # Adjust the monthly payment for the principal by subtracting the portion for added value services
        adjusted_monthlyPayment = monthlyPayment - monthly_added_value_payment
        
        # Calculate the remaining balance after making LoanTerm_months_rounded - 1 payments
        remaining_balance = npf.fv(monthly_interest_rate, LoanTerm_months_rounded - 1, -adjusted_monthlyPayment, principal)
        
        # Calculate the final payment to settle the remaining balance
        final_payment = remaining_balance * (1 + monthly_interest_rate)
        
        last_monthlyPayment = -final_payment + monthly_added_value_payment  # Adjust last payment to include the added value payment
        
        # Calculate the LoanTerm in years
        LoanTerm_years = LoanTerm_months_rounded / 12

        # Save the loan term and the last month's payment
        self.LoanTerm = LoanTerm_years
        self.last_monthlyPayment = last_monthlyPayment

        results = {
            "principal": principal,  # This is now just the principal
            "total_payment": principal + total_added_value_services,  # Total payment over the loan term
            'total_added_value': total_added_value_services,  # Total added value services
            "LoanTerm_years": LoanTerm_years,
            "LoanTerm_months": LoanTerm_months_rounded,
            "monthly_added_value_payment": monthly_added_value_payment,  # Fixed payment for added value services
            "last_monthlyPayment": last_monthlyPayment,
            "maintenance_fee": maintenance_fee,
            "warranty_fee": warranty_fee,
            "insurance_fee": insurance_fee,
            "terminal_value": terminal_value,
            "business_con_fee": business_con_fee,
            "travel_labor_cost": travel_labor_cost
        }

        return results


    def getInvoice(self, EquipmentPrice, LoanTerm, terminal_rate, insurance, maintenance, extra_warranty, bussiness_con ): 
        invoice = self.getMonthlyPayment(EquipmentPrice, LoanTerm, terminal_rate, insurance, maintenance, extra_warranty, bussiness_con)['monthlyPayment'] * LoanTerm * 12
        return invoice

    def setName(self, name):
        self.name = name

    def displayResult(self, result):
        st.subheader(f"{self.name}")
        st.write(f"{result}")

In [13]:
import pandas as pd
from parameter import *

# Initialize an empty list to store the data
data = []

# Set initial value for EquipmentPrice
a = 7500

# Loop to generate data for 200 entries
for i in range(120):
    equipment_price = a
    loan_term = get_pricing_details(equipment_price)['Loan Term']
    data.append({'EquipmentPrice': equipment_price, 'LoanTerm': loan_term})
    a += 2500

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)


In [14]:
df

,EquipmentPrice,LoanTerm
0,7500,3.0
1,10000,3.0
2,12500,3.0
3,15000,3.0
4,17500,3.0
...,...,...
115,295000,10.0
116,297500,10.0
117,300000,10.0
118,302500,10.0


In [15]:
clinic = Calculator()

list = []

for i in range(120):

    results = clinic.getMonthlyPayment(df['EquipmentPrice'][i], df['LoanTerm'][i],df['LoanTerm'][i],0)
    list.append(results['monthlyPayment'])
    
    

    

In [16]:
df['EquipmentPrice'].values

array([  7500,  10000,  12500,  15000,  17500,  20000,  22500,  25000,
        27500,  30000,  32500,  35000,  37500,  40000,  42500,  45000,
        47500,  50000,  52500,  55000,  57500,  60000,  62500,  65000,
        67500,  70000,  72500,  75000,  77500,  80000,  82500,  85000,
        87500,  90000,  92500,  95000,  97500, 100000, 102500, 105000,
       107500, 110000, 112500, 115000, 117500, 120000, 122500, 125000,
       127500, 130000, 132500, 135000, 137500, 140000, 142500, 145000,
       147500, 150000, 152500, 155000, 157500, 160000, 162500, 165000,
       167500, 170000, 172500, 175000, 177500, 180000, 182500, 185000,
       187500, 190000, 192500, 195000, 197500, 200000, 202500, 205000,
       207500, 210000, 212500, 215000, 217500, 220000, 222500, 225000,
       227500, 230000, 232500, 235000, 237500, 240000, 242500, 245000,
       247500, 250000, 252500, 255000, 257500, 260000, 262500, 265000,
       267500, 270000, 272500, 275000, 277500, 280000, 282500, 285000,
      

In [17]:
list

[290.7739902630504,
 379.3653203507339,
 467.95665043841734,
 556.5479805261008,
 645.1393106137843,
 733.7306407014678,
 555.5000729102896,
 614.4445254558774,
 673.388978001465,
 732.3334305470529,
 791.2778830926406,
 850.2223356382284,
 909.166788183816,
 968.1112407294039,
 1027.0556932749917,
 1086.0001458205793,
 1144.944598366167,
 1203.8890509117548,
 1001.7684718463751,
 1048.281256220012,
 1094.7940405936488,
 1141.3068249672858,
 1187.8196093409229,
 1234.3323937145597,
 1280.8451780881965,
 1327.3579624618335,
 1373.8707468354703,
 1420.383531209107,
 1466.8963155827444,
 1513.4090999563812,
 1559.9218843300182,
 1606.434668703655,
 1652.9474530772918,
 1699.4602374509288,
 1745.9730218245656,
 1792.4858061982027,
 1838.9985905718395,
 1885.5113749454763,
 1563.9105544598533,
 1601.444958227167,
 1638.9793619944805,
 1676.5137657617938,
 1714.0481695291073,
 1751.5825732964208,
 1789.1169770637343,
 1826.6513808310478,
 1864.1857845983614,
 1901.7201883656746,
 1939.254592

In [18]:
import seaborn as sns
import matplotlib.pyplot as plt

df['payment'] = list

# create sns line plot of list 
plt.figure(figsize=(10, 6))
sns.lineplot(data = df, x = 'EquipmentPrice', y='payment')
plt.show()

In [19]:
import seaborn as sns
import matplotlib.pyplot as plt

df['payment'] = list

# create sns line plot of list 
plt.figure(figsize=(10, 6))
sns.lineplot(data = df, x = 'EquipmentPrice', y='payment')

plt.axvline(x=df['EquipmentPrice'].iloc[5], color='red', linestyle='--', label='Loan Term = 2 yrs')
plt.axvline(x=df['EquipmentPrice'].iloc[10], color='red', linestyle='--', label='Loan Term = 2 yrs')
plt.axvline(x=df['EquipmentPrice'].iloc[20], color='red', linestyle='--', label='Loan Term = 2 yrs')

plt.xlabel('Equipment Price')
plt.ylabel('Monthly Payment')
# plt.legend()
plt.savefig('test.png')

### Generate Random DF

In [1]:
import streamlit as st
import warnings
import locale
locale.setlocale(locale.LC_ALL, 'en_GB.UTF-8')
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import numpy_financial as npf
import requests
import math
from loan_amortization import loan_amortization, loan_amortization_custom_payment
from parameter import *



class Calculator():
    def __init__(self):
        
        

        
        
        # COMMENT THIS IF FLASK CONNECTION IS WORKING
        self.cpi = 0.12
        self.markup_percentage = 0.001
        self.maintenance_ratio = 0.08
        self.warranty_rate = 0.05
        self.insurance_rate = 0.015
        self.travel_labor_cost = 300
        self.business_con_rate = 0.02
        self.monthly_interest_rate = self.cpi/12
        # self.monthly_interest_rate = ((1+self.cpi)**(1/12))-1
        
        # UNCOMMENT THIS IF FLASK CONNECTION IS WORKING
        response = requests.get("http://127.0.0.1:5000/get_parameters")
        if response.status_code == 200:
            params = response.json()
            self.cpi = params['cpi']
            self.markup_percentage = params['markup_percentage']
            self.maintenance_ratio = params['maintenance_ratio']
            self.warranty_rate = params['warranty_rate']
            self.insurance_rate = params['insurance_rate']
            self.travel_labor_cost = params['travel_labor_cost']
            self.business_con_rate = params['business_con_rate']
        else:
            st.error("Failed to load parameters!")
            
        self.name = None
        self.monthlyPayment = None
        self.totalPayment = None
        self.invoice = None
        
        
    def get_parameter(self, EquipmentPrice):
        """
        Finds the matching row where EquipmentPrice is between Bottom Price and Upper Price
        and returns a dictionary of specified columns.

        Parameters:
        EquipmentPrice (float): The price of the equipment.
        df (DataFrame): The DataFrame containing the equipment classes.

        Returns:
        dict: A dictionary with keys 'Loan Term', 'Markup Price', 'Warranty',
            'Maintenance', 'Insurance', and 'Business Con'.
        """
        
        param_config = pd.read_csv('best_config_1.csv')
        param_config = param_config.iloc[:,1:]      
        
        # Find the matching row
        matching_row = param_config[(param_config['Bottom Price'] <= EquipmentPrice) & (EquipmentPrice < param_config['Upper Price'])]
        
        if matching_row.empty:
            return {}  # or raise an exception if preferred

        # Extract the first matching row
        row = matching_row.iloc[0]
        
        # Create the result dictionary
        result = {
            'Loan Term': row['Loan Term'],
            'Markup Price': row['Markup Price'] / 100,
            'Warranty': row['Warranty'] / 100,
            'Maintenance': row['Maintenance'] / 100,
            'Insurance': row['Insurance'] / 100,
            'Business Con': row['Business Con'] / 100
        }
    
        return result

                 
    # def set_parameter(self, EquipmentPrice):
        
    #     result = self.get_parameter(EquipmentPrice)
        
    #     self.markup_percentage = result['Markup Price']
    #     self.maintenance_ratio = result['Maintenance']
    #     self.warranty_rate = result['Warranty']
    #     self.insurance_rate = result['Insurance']
    #     self.business_con_rate = result['Business Con']


    def getMonthlyPayment(self, EquipmentPrice, LoanTerm, terminal_rate, warranty_yrs, insurance='Yes', maintenance='Yes', extra_warranty=0, bussiness_con='Yes'): 
        
        
        # result = self.get_parameter(EquipmentPrice)
        
        # self.markup_percentage = result['Markup Price']
        # self.maintenance_ratio = result['Maintenance']
        # self.warranty_rate = result['Warranty']
        # self.insurance_rate = result['Insurance']
        # self.business_con_rate = result['Business Con']
        
        
        markup_price = EquipmentPrice # as it has been mark upped in the input form
        principal = markup_price
        additional_warranty = extra_warranty
        # maintenance_fee = (markup_price * self.maintenance_ratio if markup_price > 2500 else 0) if maintenance == 'Yes' else 0 
        maintenance_fee = (markup_price * self.maintenance_ratio * (warranty_yrs + additional_warranty)) if maintenance == 'Yes' else 0 
        # print(maintenance_fee)
 
        # warranty_yrs = 1 if markup_price <= 2000 else 2 if markup_price <= 5000 else 3 if markup_price <= 10000 else 5 if markup_price <= 30000 else 10
        
        warranty_fee = markup_price * self.warranty_rate * (additional_warranty)
        insurance_fee = markup_price * self.insurance_rate * (warranty_yrs + additional_warranty) if insurance == 'Yes' else 0
        travel_labor_cost = self.travel_labor_cost * LoanTerm
        business_con_fee = (markup_price * self.business_con_rate * (warranty_yrs + additional_warranty) if insurance == 'Yes' else 0) if bussiness_con == 'Yes' else 0
        
        
        total_added_value_services = maintenance_fee + warranty_fee + insurance_fee + business_con_fee + travel_labor_cost
        total_payment = total_added_value_services + principal
        
        terminal_value = markup_price * terminal_rate 
        
        
        monthly_interest_rate = self.monthly_interest_rate
        monthly_added_value_payment = total_added_value_services / (LoanTerm*12)
        monthlyPayment = npf.pmt(monthly_interest_rate, LoanTerm*12, -principal) + monthly_added_value_payment

        self.monthlyPayment = monthlyPayment
        
        results = {
            "principal": principal,
            "total_payment": total_payment, # principal + total_added_value_services
            'total_added_value': total_added_value_services,
            "monthlyPayment": monthlyPayment,
            "maintenance_fee": maintenance_fee,
            "warranty_fee": warranty_fee,
            "insurance_fee": insurance_fee,
            "business_con_fee": business_con_fee,
            "terminal_value": terminal_value,
            'travel_labor_cost': travel_labor_cost,
            'annual_rate': self.cpi,
            'loan_term': LoanTerm,
        }

        return results
    
    def getLoanTerm(self, EquipmentPrice, monthlyPayment, terminal_rate, warranty_yrs, insurance='Yes', maintenance='Yes', extra_warranty=0, bussiness_con='Yes'): 
        # Calculating the markup price (principal)
        markup_price = EquipmentPrice  # as it has been marked up in the input form
        principal = markup_price
        
        additional_warranty = extra_warranty
        
        # Calculating the maintenance fee
        maintenance_fee = (markup_price * self.maintenance_ratio * (warranty_yrs + additional_warranty)) if maintenance == 'Yes' else 0 
        
        # Calculating the warranty fee
        
        warranty_fee = markup_price * self.warranty_rate * (additional_warranty)
        
        # Calculating the insurance fee
        insurance_fee = markup_price * self.insurance_rate * (warranty_yrs + additional_warranty) if insurance == 'Yes' else 0
        
        # Travel labor cost
        travel_labor_cost = self.travel_labor_cost * (warranty_yrs + additional_warranty)
        
        # Calculating the terminal value
        terminal_value = markup_price * terminal_rate
        
        # Calculating the business continuity fee
        business_con_fee = markup_price * self.business_con_rate * (warranty_yrs + additional_warranty) if bussiness_con == 'Yes' else 0

        # Total added value services (maintenance, insurance, etc.), NOT affected by interest
        total_added_value_services = maintenance_fee + warranty_fee + insurance_fee + business_con_fee + travel_labor_cost
        
        # Define monthly interest rate
        monthly_interest_rate = self.monthly_interest_rate
        

        
        # First, calculate the LoanTerm in months based on the principal only
        LoanTerm_months = npf.nper(monthly_interest_rate, -monthlyPayment, principal, 0)
        
        # Round up to the nearest whole number of months
        LoanTerm_months_rounded = math.ceil(LoanTerm_months)
        
        # Calculate the remaining balance after making LoanTerm_months_rounded - 1 payments
        remaining_balance = npf.fv(monthly_interest_rate, LoanTerm_months_rounded - 1, -monthlyPayment, principal)
        
        # Calculate the final payment to settle the remaining balance
        final_payment = remaining_balance * (1 + monthly_interest_rate)
        
        remaining_added_value_services = total_added_value_services - ( monthlyPayment - (-final_payment) )
        
        while remaining_added_value_services > monthlyPayment:
            
            remaining_added_value_services -= monthlyPayment
            LoanTerm_months_rounded += 1

        
        last_monthlyPayment = remaining_added_value_services  # Adjust last payment to include the added value payment
        
        LoanTerm_months_rounded += 1
        
        # Calculate the LoanTerm in years
        LoanTerm_years = LoanTerm_months_rounded / 12

        # Save the loan term and the last month's payment
        self.LoanTerm = LoanTerm_years
        self.last_monthlyPayment = last_monthlyPayment

        results = {
            "principal": principal,  # This is now just the principal
            "total_payment": principal + total_added_value_services,  # Total payment over the loan term
            'total_added_value': total_added_value_services,  # Total added value services
            "LoanTerm_years": LoanTerm_years,
            "LoanTerm_months": LoanTerm_months_rounded,
            # "monthly_added_value_payment": monthly_added_value_payment,  # Fixed payment for added value services
            "last_monthlyPayment": last_monthlyPayment,
            "maintenance_fee": maintenance_fee,
            "warranty_fee": warranty_fee,
            "insurance_fee": insurance_fee,
            "terminal_value": terminal_value,
            "business_con_fee": business_con_fee,
            "travel_labor_cost": travel_labor_cost,
            'annual_rate': self.cpi,
            'loan_term': LoanTerm_years,
            'monthlyPayment': monthlyPayment,
        }

        return results


    def getInvoice(self, EquipmentPrice, LoanTerm, terminal_rate, insurance, maintenance, extra_warranty, bussiness_con ): 
        invoice = self.getMonthlyPayment(EquipmentPrice, LoanTerm, terminal_rate, insurance, maintenance, extra_warranty, bussiness_con)['monthlyPayment'] * LoanTerm * 12
        return invoice

    def setName(self, name):
        self.name = name

    def displayResult(self, result):
        st.subheader(f"{self.name}")
        st.write(f"{result}")

In [1]:
import random

In [2]:
import pandas as pd

# Define the data as a dictionary with numerical representations
data = {
    'Class': [1, 2, 3, 4, 5],
    'Bottom Price': [5000, 20000, 50000, 100000, 200000],
    'Upper Price': [20000, 50000, 100000, 200000, 1000000],
    'Loan Term': [3, 5, 7, 7, 7],
    'Markup Price': [5.0, 4.0, 2.0, 1.0, 1.0],
    'Warranty': [10.0, 5.0, 3.0, 1.0, 1.0],
    'Maintenance': [15.0, 10.0, 8.0, 5.0, 5.0],
    'Insurance': [10.0, 7.0, 5.0, 1.5, 1.5],
    'Business Con': [10.0, 5.0, 3.0, 2.0, 2.0],
    # 'Terminal Value': [5.0, 5.0, 3.0, 1.0, 1.0]
}

# Create a DataFrame
df_numeric = pd.DataFrame(data)



In [3]:
def generate_random_data_with_ranges(df, column_ranges):
    """
    Generate random integer data for the DataFrame based on specified column ranges.
    Each column must select its values from a predefined list in the column_ranges dictionary.
    Columns with a specified list will follow the sort rule:
    - Non-decreasing for 'Loan Term' (formerly 'Free Warranty Years')
    - Non-increasing for all other columns.

    Parameters:
        df (pd.DataFrame): The original DataFrame containing the data.
        column_ranges (dict): A dictionary specifying the allowed values for each column.
    
    Returns:
        pd.DataFrame: DataFrame with randomly generated integer values ordered based on rules.
    """
    
    # Create a copy of the dataframe to avoid modifying the original
    df_random = df.copy()

    # Iterate through each column and generate random values from the list in column_ranges
    for column, value_range in column_ranges.items():
        # Randomly select values from the provided range for each column
        random_values = [random.choice(value_range) for _ in range(5)]
        
        # Apply sorting rules based on the column
        if column == 'Loan Term':
            # Sort 'Loan Term' in non-decreasing order (ascending)
            random_values.sort()
        else:
            # Sort other columns in non-increasing order (descending)
            random_values.sort(reverse=True)
        
        # Assign the sorted values back to the DataFrame
        df_random[column] = random_values

    return df_random




In [4]:
# Define a dictionary with column names as keys and allowed values as lists
import random


column_ranges = {
    'Loan Term': [2, 3, 5, 7],
    'Markup Price': [1, 3, 5, 7, 9],
    'Warranty': [2, 4, 6, 8, 10],
    'Maintenance': [5, 10, 12, 15],
    'Insurance': [1, 2, 4, 5, 7, 10],
    'Business Con': [1, 3, 5, 7, 10],
    # 'Terminal Value': [1, 2, 3, 4, 5]
    }


df_random = generate_random_data_with_ranges(df_numeric, column_ranges)

In [9]:
df_random

,Class,Bottom Price,Upper Price,Loan Term,Markup Price,Warranty,Maintenance,Insurance,Business Con
0,1,5000,20000,2,9,10,15,7,10
1,2,20000,50000,3,9,8,12,4,5
2,3,50000,100000,3,3,8,12,2,3
3,4,100000,200000,3,1,8,5,2,3
4,5,200000,1000000,5,1,4,5,2,1


In [5]:
def get_pricing_details_from_df(equipment_price, df):
    """
    Function to get pricing details based on the equipment price from a given DataFrame.
    
    Parameters:
        equipment_price (float): The price of the equipment.
        df (pd.DataFrame): The DataFrame containing pricing data.
        
    Returns:
        dict: A dictionary containing the pricing details for the corresponding class.
    """
    
    # Find the row where the equipment price falls between the bottom and upper price
    row = df[(df['Bottom Price'] < equipment_price) & (df['Upper Price'] >= equipment_price)]
    
    # If no class matches, return None
    if row.empty:
        return None
    
    # Convert the row to a dictionary and return
    pricing_details = row.iloc[0].to_dict()
    return pricing_details

In [11]:
get_pricing_details_from_df(30000,df_random)

{'Class': 2,
 'Bottom Price': 20000,
 'Upper Price': 50000,
 'Loan Term': 3,
 'Markup Price': 9,
 'Warranty': 8,
 'Maintenance': 12,
 'Insurance': 4,
 'Business Con': 5}

### Running on random df generation

In [12]:
df_random

,Class,Bottom Price,Upper Price,Loan Term,Markup Price,Warranty,Maintenance,Insurance,Business Con
0,1,5000,20000,2,9,10,15,7,10
1,2,20000,50000,3,9,8,12,4,5
2,3,50000,100000,3,3,8,12,2,3
3,4,100000,200000,3,1,8,5,2,3
4,5,200000,1000000,5,1,4,5,2,1


In [13]:
import pandas as pd
from parameter import *

# Initialize an empty list to store the data
data = []

# Set initial value for EquipmentPrice
a = 7500

# Loop to generate data for 200 entries
for i in range(100):
    equipment_price = a
    loan_term = get_pricing_details_from_df(equipment_price,df_random)['Loan Term']
    data.append({'EquipmentPrice': equipment_price, 'LoanTerm': loan_term})
    a += 2500

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)


In [14]:
from single_calculator import Calculator

clinic = Calculator()

list = []

for i in range(100):
    
    

    results = clinic.getMonthlyPayment(df['EquipmentPrice'][i], df['LoanTerm'][i],df['LoanTerm'][i],0)
    list.append(results['monthlyPayment'])

In [15]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# df['payment'] = list

# # create sns line plot of list 
# plt.figure(figsize=(10, 6))
# sns.lineplot(data = df, x = 'EquipmentPrice', y='payment')

# plt.axvline(x=df['EquipmentPrice'].iloc[3], color='red', linestyle='--', label='Loan Term = 2 yrs')
# plt.axvline(x=df['EquipmentPrice'].iloc[9], color='red', linestyle='--', label='Loan Term = 2 yrs')
# plt.axvline(x=df['EquipmentPrice'].iloc[19], color='red', linestyle='--', label='Loan Term = 2 yrs')

# plt.xlabel('Equipment Price')
# plt.ylabel('Monthly Payment')
# # plt.legend()
# plt.savefig('test.png')

### Build as function

In [6]:
import pandas as pd
from parameter import *
import seaborn as sns
import matplotlib.pyplot as plt
from parameter import *

In [7]:
# Define a dictionary with column names as keys and allowed values as lists
# column_ranges = {
#     'Loan Term': [2, 3, 5, 7],
#     'Markup Price': [1, 3, 5, 7, 9],
#     'Warranty': [2, 4, 6, 8, 10],
#     'Maintenance': [5, 7, 10],
#     'Insurance': [1, 2, 3, 5, 7, 10],
#     'Business Con': [1, 3, 5, 7],
# }

column_ranges = {
    'Loan Term': [2, 3, 5, 7],
    'Markup Price': [1, 3, 5, 7, 9],
    'Warranty': [2, 4, 6, 8],
    'Maintenance': [5, 7, 9],
    'Insurance': [1, 2, 3, 5],
    'Business Con': [1, 3, 5, 7],
}


# column_ranges = {
#     'Loan Term': [2, 3, 5, 7],
#     'Markup Price': [1, 3],
#     'Warranty': [1, 4],
#     'Maintenance': [1,3],
#     'Insurance': [1, 2],
#     'Business Con': [1, 3],
# }

In [13]:
from single_calculator import Calculator

def run_random_sim():

    # randomize parameter configuration
    
    
    df_random = generate_random_data_with_ranges(df_numeric, column_ranges)

    # Initialize an empty list to store the data
    data = []
    params = []

    # Set initial value for EquipmentPrice
    a = 7500

    # Loop to generate data for 200 entries
    for i in range(120):
        equipment_price = a
        response = get_pricing_details_from_df(equipment_price,df_random)
        loan_term = response['Loan Term']
        free_warranty = 1 if a <= 75000 else 2
        data.append({'EquipmentPrice': equipment_price, 'LoanTerm': loan_term, 'Free Warranty': free_warranty})
        
        params.append({
            'Markup Price': response['Markup Price'],
            'Warranty': response['Warranty'],
            'Maintenance': response['Maintenance'],
            'Insurance': response['Insurance'],
            'Business Con': response['Business Con'],
            # 'Terminal Value': response['Terminal Value']
            
        })
        
        a += 2500

    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(data)
    
    clinic = Calculator()

    list = []
    total_payment = []
    finance_product = []
    maintenance_fee = []
    travel_labor_cost = []
    insurance_fee = []
    warranty_fee = []
    bus_con_fee = []
    

    for i in range(120):
        
        clinic.markup_percentage = params[i]['Markup Price'] / 100
        clinic.warranty_rate = params[i]['Warranty'] / 100
        clinic.maintenance_ratio = params[i]['Maintenance'] / 100
        clinic.insurance_rate = params[i]['Insurance'] / 100
        clinic.business_con_rate = params[i]['Business Con'] / 100
        
        # clinic.markup_percentage = 0
        # clinic.warranty_rate = 0
        # clinic.maintenance_ratio = 0
        # clinic.insurance_rate = 0
        # clinic.business_con_rate = 0
        
        
        warranty = 1 if df['EquipmentPrice'][i] <= 75000 else 2
        terminal_rate = 0.1
        
        # print(clinic.warranty_rate, clinic.maintenance_ratio, clinic.insurance_rate, clinic.business_con_rate)
        
        extra_warranty = 2 if df['EquipmentPrice'][i] <= 75000 else 1
        
        
        results = clinic.getMonthlyPayment(EquipmentPrice=df['EquipmentPrice'][i], LoanTerm=df['LoanTerm'][i],warranty_yrs=warranty, extra_warranty=extra_warranty, insurance='Yes')
        list.append(results['monthlyPayment'])
        
        
        total_payment.append(results['monthlyPayment'] * (df['LoanTerm'][i]*12))
        
        
        
        finance_product_profit = results['maintenance_fee'] + results['warranty_fee'] + results['insurance_fee'] + results['business_con_fee'] +  results['travel_labor_cost']
        
        finance_product.append(finance_product_profit)
        maintenance_fee.append(results['maintenance_fee'])
        travel_labor_cost.append(results['travel_labor_cost'])
        insurance_fee.append(results['insurance_fee'])
        warranty_fee.append(results['warranty_fee'])
        bus_con_fee.append(results['business_con_fee'])
        
        
    df['payment'] = list
    df['total_payment'] = total_payment
    df['finance_product_gain'] = finance_product
    df['maintenance_fee'] = maintenance_fee
    df['travel_labor'] = travel_labor_cost
    df['insurance'] = insurance_fee
    df['warranty_fee'] = warranty_fee
    df['buscon_fee'] = bus_con_fee
    df['total_interest'] = df['total_payment'] - df['EquipmentPrice'] - df['finance_product_gain']
    df['total_gain']  = df['total_payment'] - df['EquipmentPrice']
    
    df['total_period'] = np.where(df['LoanTerm'] >= df['additional_warranty'] + df['Free Warranty'], 
                              df['LoanTerm'], 
                              df['additional_warranty'] + df['Free Warranty'])
    
    df['yearly_added_value_ratio'] = df['finance_product_gain'] / df['total_period'] / df['EquipmentPrice']
    

    # # create sns line plot of list 
    # plt.figure(figsize=(10, 6))
    # sns.lineplot(data = df, x = 'EquipmentPrice', y='payment')

    # plt.axvline(x=df['EquipmentPrice'].iloc[6], color='red', linestyle='--', label='Loan Term = 2 yrs')
    # plt.axvline(x=df['EquipmentPrice'].iloc[18], color='red', linestyle='--', label='Loan Term = 2 yrs')
    # plt.axvline(x=df['EquipmentPrice'].iloc[38], color='red', linestyle='--', label='Loan Term = 2 yrs')
    # plt.axvline(x=df['EquipmentPrice'].iloc[78], color='red', linestyle='--', label='Loan Term = 2 yrs')

    # plt.xlabel('Equipment Price')
    # plt.ylabel('Monthly Payment')
    # plt.legend()
    # plt.show()
    # plt.savefig('test.png')
    
    return df_random, df['total_gain'].mean(), df
        


In [14]:
df, avg_gain, output_df = run_random_sim()
df

,Class,Bottom Price,Upper Price,Loan Term,Markup Price,Warranty,Maintenance,Insurance,Business Con
0,1,5000,20000,3,9,8,9,5,7
1,2,20000,50000,5,3,6,9,5,3
2,3,50000,100000,5,3,6,9,3,3
3,4,100000,200000,7,1,4,9,3,1
4,5,200000,1000000,7,1,2,7,3,1


In [17]:
output_df.head(10)

,EquipmentPrice,LoanTerm,payment,total_payment,finance_product_gain,maintenance_fee,travel_labor,insurance,warranty_fee,buscon_fee,total_interest,total_gain,yearly_added_value_ratio
0,7500,3,438.690657,15792.863649,6825.0,2025.0,900,1125.0,1200.0,1575.0,1467.863649,8292.863649,0.303333
1,10000,3,576.587543,20757.151533,8800.0,2700.0,900,1500.0,1600.0,2100.0,1957.151533,10757.151533,0.293333
2,12500,3,714.484428,25721.439416,10775.0,3375.0,900,1875.0,2000.0,2625.0,2446.439416,13221.439416,0.287333
3,15000,3,852.381314,30685.727299,12750.0,4050.0,900,2250.0,2400.0,3150.0,2935.727299,15685.727299,0.283333
4,17500,3,990.278200,35650.015182,14725.0,4725.0,900,2625.0,2800.0,3675.0,3425.015182,18150.015182,0.280476
5,20000,3,1072.619530,38614.303065,14700.0,5400.0,900,1800.0,2400.0,4200.0,3914.303065,18614.303065,0.245000
6,22500,5,926.750073,55605.004375,25575.0,10125.0,1500,3375.0,2700.0,7875.0,7530.004375,33105.004375,0.227333
7,25000,5,1026.944525,61616.671527,28250.0,11250.0,1500,3750.0,3000.0,8750.0,8366.671527,36616.671527,0.226000
8,27500,5,1127.138978,67628.338680,30925.0,12375.0,1500,4125.0,3300.0,9625.0,9203.338680,40128.338680,0.224909
9,30000,5,1227.333431,73640.005833,33600.0,13500.0,1500,4500.0,3600.0,10500.0,10040.005833,43640.005833,0.224000


In [15]:
# Monthly payment constraint DataFrame
monthly_payment_constraint = pd.DataFrame({
    'Class': [1, 2, 3, 4, 5],
    'Bottom Price': [5000, 20000, 50000, 100000, 200000],
    'Upper Price': [20000, 50000, 100000, 200000, 1000000],
    'Lowest Payment': [300, 500, 750, 1500, 2000],
    # 'Highest Payment': [2000, 3500, 5000, 7000, 9000]
    'Highest Payment': [1500, 2500, 4000, 6000, 8000]
})

# minimum_added_value = 0.18
# maximum_added_value = 0.22

# List to store the top 5 best results
top_results = []

# Initialize counters for infeasible conditions
num_infeasible_too_low = 0
num_infeasible_too_high = 0
num_infeasible_added_value_ratio = 0

# Define a function to update the top 5 results
def update_top_results(avg_gain, df, config, top_results):
    # If the top_results list has less than 5 entries, add the new result
    if len(top_results) < 5:
        top_results.append((avg_gain, df, config))
    else:
        # Check if the new avg_gain is better than the worst in the top 5
        min_avg_gain = min(top_results, key=lambda x: x[0])[0]
        if avg_gain > min_avg_gain:
            # Replace the worst result with the new one
            min_index = top_results.index(min(top_results, key=lambda x: x[0]))
            top_results[min_index] = (avg_gain, df, config)

    # Sort the list by avg_gain in descending order
    top_results.sort(key=lambda x: x[0], reverse=True)

for i in range(5):
    df_random, avg_gain, df = run_random_sim()  # Get the DataFrame from simulation

    # Check for each row if the constraints are met
    all_constraints_satisfied = True
    for idx, row in df.iterrows():
        # Find the corresponding row in monthly_payment_constraint based on EquipmentPrice
        matching_constraint = monthly_payment_constraint[
            (monthly_payment_constraint['Bottom Price'] <= row['EquipmentPrice']) & 
            (monthly_payment_constraint['Upper Price'] > row['EquipmentPrice'])
        ]
        
        # If no matching constraint found, skip this entry
        if matching_constraint.empty:
            all_constraints_satisfied = False
            break
        
        # Extract the lowest and highest payment bounds
        lowest_payment = matching_constraint['Lowest Payment'].values[0]
        highest_payment = matching_constraint['Highest Payment'].values[0]
        
        # Check if the payment is within bounds
        if row['payment'] < lowest_payment:
            all_constraints_satisfied = False
            num_infeasible_too_low += 1  # Increment counter for too low payments
            break
        elif row['payment'] >= highest_payment:
            all_constraints_satisfied = False
            num_infeasible_too_high += 1  # Increment counter for too high payments
            break
        
        # if row['finance_product_gain'] / row['LoanTerm'] / row['EquipmentPrice'] < minimum_added_value:
        #     num_infeasible_added_value_ratio += 1
        #     break
        # if row['finance_product_gain'] / row['LoanTerm']/ row['EquipmentPrice'] > maximum_added_value:
        #     num_infeasible_added_value_ratio += 1
        #     break

    # If all constraints are satisfied and the average gain is good, consider for top results
    if all_constraints_satisfied:
        update_top_results(avg_gain, df, df_random, top_results)

# After the loop, top_results will contain the best 5 configurations sorted by avg_gain
# top_results is a list of tuples: [(avg_gain, best_df, best_config), ...]

# You can access them like this:
for rank, (gain, df_result, config_result) in enumerate(top_results, start=1):
    print(f"Rank {rank}: Average Gain = {gain}")
    # Optionally, you can inspect df_result and config_result


In [16]:
print(num_infeasible_too_low)
print(num_infeasible_too_high)
print(num_infeasible_added_value_ratio)

0
5
0


In [22]:
# the extreme case
best_avg_gain = top_results[0][0]   
best_df = top_results[0][1]         
best_config = top_results[0][2]     

top_results[0][2] 

,Class,Bottom Price,Upper Price,Loan Term,Markup Price,Warranty,Maintenance,Insurance,Business Con
0,1,5000,20000,3,9,8,9,5,5
1,2,20000,50000,5,7,4,7,5,5
2,3,50000,100000,5,5,4,5,5,3
3,4,100000,200000,7,5,4,5,3,3
4,5,200000,1000000,7,1,2,5,2,1


In [23]:
best_df

,EquipmentPrice,LoanTerm,payment,total_payment,finance_product_gain,maintenance_fee,travel_labor,insurance,warranty_fee,buscon_fee,total_interest,total_gain,yearly_added_value_ratio
0,7500,3,438.690657,15792.863649,6825.0,2025.0,900,1125.0,1200.0,1575.0,1467.863649,8292.863649,0.303333
1,10000,3,576.587543,20757.151533,8800.0,2700.0,900,1500.0,1600.0,2100.0,1957.151533,10757.151533,0.293333
2,12500,3,714.484428,25721.439416,10775.0,3375.0,900,1875.0,2000.0,2625.0,2446.439416,13221.439416,0.287333
3,15000,3,852.381314,30685.727299,12750.0,4050.0,900,2250.0,2400.0,3150.0,2935.727299,15685.727299,0.283333
4,17500,3,990.278200,35650.015182,14725.0,4725.0,900,2625.0,2800.0,3675.0,3425.015182,18150.015182,0.280476
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,295000,7,7655.770461,643084.718711,205650.0,103250.0,2100,20650.0,17700.0,61950.0,142434.718711,348084.718711,0.099588
116,297500,7,7720.438007,648516.792599,207375.0,104125.0,2100,20825.0,17850.0,62475.0,143641.792599,351016.792599,0.099580
117,300000,7,7785.105553,653948.866486,209100.0,105000.0,2100,21000.0,18000.0,63000.0,144848.866486,353948.866486,0.099571
118,302500,7,7849.773100,659380.940374,210825.0,105875.0,2100,21175.0,18150.0,63525.0,146055.940374,356880.940374,0.099563


In [109]:
# Access the second best result
best_avg_gain = top_results[0][0]   
best_df = top_results[0][1]         
best_config = top_results[0][2]     

top_results[0][2] 

,Class,Bottom Price,Upper Price,Loan Term,Markup Price,Warranty,Maintenance,Insurance,Business Con
0,1,5000,20000,2,7,8,9,7,7
1,2,20000,50000,5,7,8,7,7,7
2,3,50000,100000,7,7,8,7,5,7
3,4,100000,200000,7,7,6,5,5,7
4,5,200000,1000000,7,1,2,5,2,5


In [110]:
best_df.head(10)

,EquipmentPrice,LoanTerm,payment,total_payment,finance_product_gain,maintenance_fee,travel_labor,insurance,warranty_fee,buscon_fee,total_interest,total_gain
0,7500,2,449.926042,10798.225000,2325.0,675.0,600,525.0,0.0,525.0,973.225000,3298.225000
1,10000,2,591.568056,14197.633334,2900.0,900.0,600,700.0,0.0,700.0,1297.633334,4197.633334
2,12500,2,733.210069,17597.041667,3475.0,1125.0,600,875.0,0.0,875.0,1622.041667,5097.041667
3,15000,2,874.852083,20996.450000,4050.0,1350.0,600,1050.0,0.0,1050.0,1946.450000,5996.450000
4,17500,2,1016.494097,24395.858334,4625.0,1575.0,600,1225.0,0.0,1225.0,2270.858334,6895.858334
5,20000,2,1158.136111,27795.266667,5200.0,1800.0,600,1400.0,0.0,1400.0,2595.266667,7795.266667
6,22500,5,604.250073,36255.004375,6225.0,1575.0,1500,1575.0,0.0,1575.0,7530.004375,13755.004375
7,25000,5,668.611192,40116.671527,6750.0,1750.0,1500,1750.0,0.0,1750.0,8366.671527,15116.671527
8,27500,5,732.972311,43978.338680,7275.0,1925.0,1500,1925.0,0.0,1925.0,9203.338680,16478.338680
9,30000,5,797.333431,47840.005833,7800.0,2100.0,1500,2100.0,0.0,2100.0,10040.005833,17840.005833


In [91]:
best_df = top_results[0][1]         # The second best DataFrame (df)
best_config = top_results[0][2] 
best_config.loc[0, 'Bottom Price'] = 0

best_df.to_csv('best_df_1.csv')
best_config.to_csv('best_config_1.csv')

In [21]:
best_df = top_results[1][1]         # The second best DataFrame (df)
best_config = top_results[1][2] 
best_config.loc[0, 'Bottom Price'] = 0

best_df.to_csv('best_df_2.csv')
best_config.to_csv('best_config_2.csv')

In [22]:
best_df = top_results[2][1]         # The second best DataFrame (df)
best_config = top_results[2][2] 
best_config.loc[0, 'Bottom Price'] = 0

best_df.to_csv('best_df_3.csv')
best_config.to_csv('best_config_3.csv')

In [2]:
best_df = pd.read_csv('best_df_1.csv')
best_config = pd.read_csv('best_config_1.csv')

In [3]:
best_df

,Unnamed: 0,EquipmentPrice,LoanTerm,payment,total_payment,finance_product_gain,maintenance_fee,travel_labor,insurance,warranty_fee,buscon_fee,total_interest,total_gain
0,0,7500,2,565.863542,13580.725000,5107.5,1417.5,600,787.5,1200.0,1102.5,973.225000,6080.725000
1,1,10000,2,746.151389,17907.633334,6610.0,1890.0,600,1050.0,1600.0,1470.0,1297.633334,7907.633334
2,2,12500,2,926.439236,22234.541667,8112.5,2362.5,600,1312.5,2000.0,1837.5,1622.041667,9734.541667
3,3,15000,2,1106.727083,26561.450000,9615.0,2835.0,600,1575.0,2400.0,2205.0,1946.450000,11561.450000
4,4,17500,2,1287.014931,30888.358334,11117.5,3307.5,600,1837.5,2800.0,2572.5,2270.858334,13388.358334
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,115,295000,7,5790.949032,486439.718711,49005.0,16225.0,2100,3245.0,17700.0,9735.0,142434.718711,191439.718711
116,116,297500,7,5839.813007,490544.292599,49402.5,16362.5,2100,3272.5,17850.0,9817.5,143641.792599,193044.292599
117,117,300000,7,5888.676982,494648.866486,49800.0,16500.0,2100,3300.0,18000.0,9900.0,144848.866486,194648.866486
118,118,302500,7,5937.540957,498753.440374,50197.5,16637.5,2100,3327.5,18150.0,9982.5,146055.940374,196253.440374


### Creating Dataframe for the actual product default value

In [18]:
import pandas as pd
import numpy as np
# from single_calculator import Calculator

In [19]:
df = pd.read_csv('best_config_1.csv')
df = df.iloc[:,1:]
df.iloc[0,1] = 0
df

,Class,Bottom Price,Upper Price,Loan Term,Markup Price,Warranty,Maintenance,Insurance,Business Con
0,1,0,20000,2,9,8,9,5,7
1,2,20000,50000,3,9,6,9,3,7
2,3,50000,100000,5,7,6,5,3,7
3,4,100000,200000,7,7,6,5,1,5
4,5,200000,1000000,7,5,6,5,1,3


In [20]:
def get_pricing_details_from_df(equipment_price, df):
    """
    Function to get pricing details based on the equipment price from a given DataFrame.
    
    Parameters:
        equipment_price (float): The price of the equipment.
        df (pd.DataFrame): The DataFrame containing pricing data.
        
    Returns:
        dict: A dictionary containing the pricing details for the corresponding class.
    """
    
    # Find the row where the equipment price falls between the bottom and upper price
    row = df[(df['Bottom Price'] < equipment_price) & (df['Upper Price'] >= equipment_price)]
    
    # If no class matches, return None
    if row.empty:
        return None
    
    # Convert the row to a dictionary and return
    pricing_details = row.iloc[0].to_dict()
    return pricing_details

In [28]:
from single_calculator import *


def run_best_sim(df):

    # randomize parameter configuration
    
    
    df_random = df

    # Initialize an empty list to store the data
    data = []
    params = []

    # Set initial value for EquipmentPrice
    equipment_df = pd.read_excel('Input Streamlit v3.xlsx')

    # Loop to generate data for 200 entries
    for i in range(len(equipment_df)):
        equipment_price = equipment_df['Price'].iloc[i]
        equipment_name = equipment_df['Product Name'].iloc[i]
        equipment_type = equipment_df['Type'].iloc[i]
        response = get_pricing_details_from_df(equipment_price,df_random)
        loan_term = response['Loan Term']
        free_warranty = equipment_df['Warranty (years)'].iloc[i]
        data.append({'Equipment Name': equipment_name, 'Type': equipment_type,'EquipmentPrice': equipment_price, 'LoanTerm': loan_term, 'Free Warranty': free_warranty})
        
        params.append({
            'Markup Price': response['Markup Price'],
            'Warranty': response['Warranty'],
            'Maintenance': response['Maintenance'],
            'Insurance': response['Insurance'],
            'Business Con': response['Business Con'],
            # 'Terminal Value': response['Terminal Value']
            
        })

    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(data)
    
    clinic = Calculator()

    list = []
    total_payment = []
    finance_product = []
    maintenance_fee = []
    travel_labor_cost = []
    insurance_fee = []
    warranty_fee = []
    bus_con_fee = []
    

    for i in range(len(equipment_df)):
        
        clinic.markup_percentage = params[i]['Markup Price'] / 100
        clinic.warranty_rate = params[i]['Warranty'] / 100
        clinic.maintenance_ratio = params[i]['Maintenance'] / 100
        clinic.insurance_rate = params[i]['Insurance'] / 100
        clinic.business_con_rate = params[i]['Business Con'] / 100
        
        # clinic.markup_percentage = 0
        # clinic.warranty_rate = 0
        # clinic.maintenance_ratio = 0
        # clinic.insurance_rate = 0
        # clinic.business_con_rate = 0
        
        
        warranty = 1 if df['EquipmentPrice'][i] <= 75000 else 2
        terminal_rate = 0.1
        
        # print(clinic.warranty_rate, clinic.maintenance_ratio, clinic.insurance_rate, clinic.business_con_rate)
        
        extra_warranty = 2 if df['EquipmentPrice'][i] <= 75000 else 1
        
        
        results = clinic.getMonthlyPayment(EquipmentPrice=df['EquipmentPrice'][i], LoanTerm=df['LoanTerm'][i],warranty_yrs=warranty, extra_warranty=extra_warranty, insurance='Yes')
        list.append(results['monthlyPayment'])
        
        
        
        total_payment.append(results['monthlyPayment'] * (df['LoanTerm'][i]*12))
        
        
        
        finance_product_profit = results['maintenance_fee'] + results['warranty_fee'] + results['insurance_fee'] + results['business_con_fee'] +  results['travel_labor_cost']
        
        
        finance_product.append(finance_product_profit)
        maintenance_fee.append(results['maintenance_fee'])
        travel_labor_cost.append(results['travel_labor_cost'])
        insurance_fee.append(results['insurance_fee'])
        warranty_fee.append(results['warranty_fee'])
        bus_con_fee.append(results['business_con_fee'])
        
    
    df['additional_warranty'] = extra_warranty   
    df['payment'] = list
    df['total_payment'] = total_payment
    df['finance_product_gain'] = finance_product
    df['maintenance_fee'] = maintenance_fee
    df['travel_labor'] = travel_labor_cost
    df['insurance'] = insurance_fee
    df['warranty_fee'] = warranty_fee
    df['buscon_fee'] = bus_con_fee
    df['total_interest'] = df['total_payment'] - df['EquipmentPrice'] - df['finance_product_gain']
    df['total_gain']  = df['total_payment'] - df['EquipmentPrice']
    
    df['total_period'] = np.where(df['LoanTerm'] >= df['additional_warranty'] + df['Free Warranty'], 
                              df['LoanTerm'], 
                              df['additional_warranty'] + df['Free Warranty'])
    
    df['yearly_added_value_ratio'] = df['finance_product_gain'] / df['total_period'] / df['EquipmentPrice']
    
    
    return df_random, df['total_gain'].mean(), df
        


In [29]:
df, avg_gain, output_df = run_best_sim(df)
df

,Class,Bottom Price,Upper Price,Loan Term,Markup Price,Warranty,Maintenance,Insurance,Business Con
0,1,0,20000,2,9,8,9,5,7
1,2,20000,50000,3,9,6,9,3,7
2,3,50000,100000,5,7,6,5,3,7
3,4,100000,200000,7,7,6,5,1,5
4,5,200000,1000000,7,5,6,5,1,3


In [30]:
output_df

,Equipment Name,Type,EquipmentPrice,LoanTerm,Free Warranty,additional_warranty,payment,total_payment,finance_product_gain,maintenance_fee,travel_labor,insurance,warranty_fee,buscon_fee,total_interest,total_gain,total_period,yearly_added_value_ratio
0,Ultrasonic Cleaners,5 Litres,726,2,1,2,95.572841,2293.748180,1473.54,196.02,900,108.90,116.16,152.46,94.208180,1567.748180,3,0.676556
1,Ultrasonic Cleaners,10 Litres,1090,2,1,2,124.689251,2992.542033,1761.10,294.30,900,163.50,174.40,228.90,141.442033,1902.542033,3,0.538563
2,Bonida Hd Intraoral Camera,NaN,1636,2,1,2,168.363867,4040.732813,2192.44,441.72,900,245.40,261.76,343.56,212.292813,2404.732813,3,0.446707
3,Plant Equipment,Chair Compressor One-Chair,1817,2,1,2,182.842082,4388.209977,2335.43,490.59,900,272.55,290.72,381.57,235.779977,2571.209977,3,0.428441
4,Plant Equipment,Chair Compressor Two-Chair,2999,2,1,2,277.390427,6657.370237,3269.21,809.73,900,449.85,479.84,629.79,389.160237,3658.370237,3,0.363367
5,Plant Equipment,Chair Suction Two-Three Chair,3000,2,1,2,277.470417,6659.290000,3270.00,810.00,900,450.00,480.00,630.00,389.290000,3659.290000,3,0.363333
6,Runyes Autoclaves,12L B Autoclave,3090,2,1,2,284.669529,6832.068700,3341.10,834.30,900,463.50,494.40,648.90,400.968700,3742.068700,3,0.360421
7,Runyes Autoclaves,5L B Fast Cycle Autoclave,3635,2,1,2,328.264155,7878.339717,3771.65,981.45,900,545.25,581.60,763.35,471.689717,4243.339717,3,0.345864
8,Runyes X-Rays,Intpaopal Sensop,3635,2,1,2,328.264155,7878.339717,3771.65,981.45,900,545.25,581.60,763.35,471.689717,4243.339717,3,0.345864
9,Runyes X-Rays,Handheld X-Ray,3900,2,1,2,349.461542,8387.077000,3981.00,1053.00,900,585.00,624.00,819.00,506.077000,4487.077000,3,0.340256


In [31]:
output_df.to_csv('equipment_default_df_1.csv')